# $\texttt{GiRaFFE}$: Solving GRFFE equations at a higher Finite Differencing order

## Authors: Zach Etienne & Patrick Nelson
### Formatting improvements courtesy Brandon Clark

### NRPy+ Source Code for this module: [GiRaFFE_HO/GiRaFFE_Higher_Order_v2.py](../edit/GiRaFFE_HO/GiRaFFE_Higher_Order_v2.py) [\[tutorial\]](Tutorial-GiRaFFE_Higher_Order_v2.ipynb)  Constructs GRFFE evolution equations and initial data as SymPy expressions.

<font color='red'>**GiRaFFE_HO_v2 is an experiment that omits the analytic derivatives of $\partial_j T^j_{{\rm EM} i}$.**</font>

<font color='red'>**Note that, for full compatibility with .par files, running this notebook will *overwrite* the thorn GiraFFE_HO**</font>

## Introduction:

This module focuses on using the equations developed in the [Tutorial-GiRaFFE_Higher_Order_v2](Tutorial-GiRaFFE_Higher_Order_v2.ipynb) tutorial module to build an Einstein Toolkit (ETK) thorn to solve the GRFFE equations in Cartesian coordinates. This tutorial will focus on implementing the time evolution aspects; others can be contructed to set up specific initial data.

When interfaced properly with the ETK, this module will propagate the initial data for $\tilde{S}_i$, $A_i$, and $\sqrt{\gamma} \Phi$, defined in the last tutorial, forward in time by integrating the equations for $\partial_t \tilde{S}_i$, $\partial_t A_i$ and $\partial_t [\sqrt{\gamma} \Phi]$ subject to spatial boundary conditions. The time evolution itself is handled by the $\text{MoL}$ (Method of Lines) thorn in the $\text{CactusNumerical}$ arrangement, and the boundary conditions by the $\text{Boundary}$ thorn in the $\text{CactusBase}$ arrangement. 

Similar to the other ETK modules we have built, we will construct the WaveToyNRPy module in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the evolution equations into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$ 

This module is organized as follows

1. [Step 1](#initializenrpy): Call on NRPy+ to convert the SymPy expression for the evolution equations and A-to-B into one C-code kernel for each
    1. [Step 1.a](#import): Import NRPy+ core modules and the GiRaFFE_Higher_Order_v2 module
    1. [Step 1.b](#ccode_output): Create the C code output kernel 
    1. [Step 1.c](#ccode_write): Write C code to files
    1. [Step 1.d](#a2bdriver): The A-to-B driver
1. [Step 2](#etk): Interfacing with the Einstein Toolkit
    1. [Step 2.a](#einstein_c): Constructing the Einstein Toolkit C-code calling functions that include the C code kernels
    1. [Step 2.b](#conservative2primitive): The Conservative to Primitive Solver
        1. [Step 2.b.i](#old_giraffe): Including "GiRaFFE_headers.h" from old version of Giraffe
        1. [Step 2.b.ii](#compute_conservatives): Writing compute_conservatives_FFE.C
    1. [Step 2.c](#grmhd): GRMHD variables provided by HydroBase
    1. [Step 2.d](#timelevel): Copying initial data to additional timelevels
    1. [Step 2.e](#cclfiles): CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure
        1. [Step2.e.i](#interface): interface.ccl
        1. [Step2.e.ii](#param): param.ccl
        1. [Step2.e.iiii](#schedule): schedule.ccl
    1. [Step 2.f](#einstein_list): Add the C code to the Einstein Toolkit compilation list
1. [Step 3](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='initializenrpy'></a>

# Step 1: Call on NRPy+ to convert the SymPy expressions for the evolution equations and A-to-B into one C-code kernel for each  \[Back to [top](#toc)\]
$$\label{initializenrpy}$$




<a id='import'></a>

## Step 1.a: Import NRPy+ core modules and the GiRaFFE_Higher_Order_v2 module \[Back to [top](#toc)\]
$$\label{import}$$

We start by importing the core NRPy+ modules we need and setting commonly used parameters. Since we are writing an ETK thorn, we'll need to set "grid::GridFuncMemAccess" to "ETK". We will then import the GiRaFFE_Higher_Order.py module we previously created and run its main function to generate the SymPy for the expressions we want.
$$\label{step1p1}$$

In [1]:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

#Step 0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")

# Step 1c: Call the GiRaFFE_Higher_Order() function from within the
#          GiRaFFE_HO/GiRaFFE_Higher_Order.py module.
import GiRaFFE_HO.GiRaFFE_Higher_Order_v2 as gho
gho.GiRaFFE_Higher_Order_v2()

# Step 1: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)


<a id='ccode_output'></a>

## Step 1.b: Create the C code output kernel \[Back to [top](#toc)\]
$$\label{ccode_output}$$

To do this, for each .h file we need, we will set up an array of lhrh objects representing the gridfunctions to print. We will then pass that array to FD_outputC to get a string of optimized C code. Next, we use the loop function to add code to the kernel so that is applied to the whole ETK grid.


In [2]:
# Step 2: Create the C code output kernel.
Prereqs_to_print = [\
                   lhrh(lhs=gri.gfaccess("out_gfs","AevolParen"),rhs=gho.AevolParen),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU0"),rhs=gho.PevolParenU[0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU1"),rhs=gho.PevolParenU[1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU2"),rhs=gho.PevolParenU[2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD00"),rhs=gho.SevolParenUD[0][0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD01"),rhs=gho.SevolParenUD[0][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD02"),rhs=gho.SevolParenUD[0][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD10"),rhs=gho.SevolParenUD[1][0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD11"),rhs=gho.SevolParenUD[1][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD12"),rhs=gho.SevolParenUD[1][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD20"),rhs=gho.SevolParenUD[2][0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD21"),rhs=gho.SevolParenUD[2][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD22"),rhs=gho.SevolParenUD[2][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU00"),rhs=gho.gammaUU[0][0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU01"),rhs=gho.gammaUU[0][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU02"),rhs=gho.gammaUU[0][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU11"),rhs=gho.gammaUU[1][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU12"),rhs=gho.gammaUU[1][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU22"),rhs=gho.gammaUU[2][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammadet"),rhs=gho.gammadet),\
                   ]

metric_quantities_to_print = [\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU00"),rhs=gho.gammaUU[0][0]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU01"),rhs=gho.gammaUU[0][1]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU02"),rhs=gho.gammaUU[0][2]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU11"),rhs=gho.gammaUU[1][1]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU12"),rhs=gho.gammaUU[1][2]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU22"),rhs=gho.gammaUU[2][2]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammadet"),rhs=gho.gammadet),\
                             ]

# To best format this for the ETK, we'll need to register these gridfunctions.
Stilde_rhsD = ixp.register_gridfunctions_for_single_rank1("AUX","Stilde_rhsD")
A_rhsD = ixp.register_gridfunctions_for_single_rank1("AUX","A_rhsD")
psi6Phi_rhs = gri.register_gridfunctions("AUX","psi6Phi_rhs")
Conservs_to_print = [\
                     lhrh(lhs=gri.gfaccess("out_gfs","Stilde_rhsD0"),rhs=gho.Stilde_rhsD[0]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","Stilde_rhsD1"),rhs=gho.Stilde_rhsD[1]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","Stilde_rhsD2"),rhs=gho.Stilde_rhsD[2]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","A_rhsD0"),rhs=gho.A_rhsD[0]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","A_rhsD1"),rhs=gho.A_rhsD[1]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","A_rhsD2"),rhs=gho.A_rhsD[2]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","psi6Phi_rhs"),rhs=gho.psi6Phi_rhs),\
                    ]

Prereqs_CKernel = fin.FD_outputC("returnstring",Prereqs_to_print,params="outCverbose=False")
#Prereqs_CKernel = "const double u0 = u0GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\n" + Prereqs_CKernel
metric_quantities_CKernel = fin.FD_outputC("returnstring",metric_quantities_to_print,params="outCverbose=False")
Conservs_CKernel = fin.FD_outputC("returnstring",Conservs_to_print,params="outCverbose=False")
#Conservs_CKernel = "const double u0 = u0GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\n" + Conservs_CKernel

Prereqs_looped = loop.loop(["i2","i1","i0"],["0","0","0"],\
                           ["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                           ["1","1","1"],["#pragma omp parallel for","",""],"",\
                           Prereqs_CKernel.replace("time","cctk_time"))

metric_quantities_looped = loop.loop(["i2","i1","i0"],["0","0","0"],\
                                     ["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                                     ["1","1","1"],["#pragma omp parallel for","",""],"",\
                                     metric_quantities_CKernel.replace("time","cctk_time"))

Conservs_looped = loop.loop(["i2","i1","i0"],["cctk_nghostzones[2]","cctk_nghostzones[1]","cctk_nghostzones[0]"],\
                            ["cctk_lsh[2]-cctk_nghostzones[2]","cctk_lsh[1]-cctk_nghostzones[1]",\
                             "cctk_lsh[0]-cctk_nghostzones[0]"],\
                            ["1","1","1"],["#pragma omp parallel for","",""],"",\
                            Conservs_CKernel.replace("time","cctk_time"))


<a id='ccode_write'></a>

## Step 1.c: Write C code to files \[Back to [top](#toc)\]
$$\label{ccode_write}$$

Here, we write our C code to files after first creating appropriate directories. Note that we also import u0_smallb_Poynting__Cartesian.py for its computeu0_Cfunction.

In [3]:
# Step 3: Create directories for the thorn if they don't exist.
!mkdir GiRaFFE_HO     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir GiRaFFE_HO/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Step 4: Write the C code kernel to file.
with open("GiRaFFE_HO/src/Prereqs.h", "w") as file:
    file.write(str(Prereqs_looped))

with open("GiRaFFE_HO/src/metric_quantities.h", "w") as file:
    file.write(str(metric_quantities_looped))

with open("GiRaFFE_HO/src/Conservs.h", "w") as file:
    file.write(str(Conservs_looped))

# Step 5: Import the function to calculate u0 and write it to a file.
import u0_smallb_Poynting__Cartesian.u0_smallb_Poynting__Cartesian as u0etc
#u0etc.compute_u0_smallb_Poynting__Cartesian(gammaDD,betaU,alpha,ValenciavU,BU)

with open("GiRaFFE_HO/src/computeu0_Cfunction.h", "w") as file:
    file.write(u0etc.computeu0_Cfunction)


<a id='a2bdriver'></a>

## Steps 1.d: The A-to-B driver \[Back to [top](#toc)\]
$$\label{a2bdriver}$$

We will also need a routine to compute new Valencia 3-velocities at each timestep using a conservative-to-primitive solver. Since we need $v^i_{(n)}$ everywhere, this will require us to compute $B^i$ everywhere. However, $B^i = \epsilon^{ijk} \partial_j A_k$ requires derivatives of $A_i$, so getting $B^i$ will require some finesse. A chief aspect of this will require using lower-order finite differencing in the ghost zones. To that end, we will create .h files for each finite differencing order $\leq 10$, as well as upwinded- and downwinded-derivatives at 2nd order. These will let us compute the derivative at the outermost gridpoints.

In [4]:
# Step 6: The A-to-B driver

# Step 6a: Import the Levi-Civita symbol and build the corresponding tensor.
# We already have a handy function to define the Levi-Civita symbol in WeylScalars
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LCijk = LeviCivitaDDD[i][j][k]
            #LeviCivitaDDD[i][j][k] = LCijk * sp.sqrt(gho.gammadet)
            LeviCivitaUUU[i][j][k] = LCijk / sp.sqrt(gho.gammadet)

AD_dD = ixp.declarerank2("AD_dD","nosym")
BU = ixp.zerorank1() # BU is already registered as a gridfunction, but we need to zero its values and declare it in this scope.
# Step 6b: We can use this function to compactly reset to expressions to print at each FD order.
def set_BU_to_print():
    return [lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
            lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
            lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])]            

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

# Step 6c: We'll lower the FD order at each stage and write to a new file.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 10)
fin.FD_outputC("GiRaFFE_HO/src/B_from_A_10.h",set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 8)
fin.FD_outputC("GiRaFFE_HO/src/B_from_A_8.h",set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 6)
fin.FD_outputC("GiRaFFE_HO/src/B_from_A_6.h",set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)
fin.FD_outputC("GiRaFFE_HO/src/B_from_A_4.h",set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)
fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2.h",set_BU_to_print(),params="outCverbose=False")

# Step 6c: For the outermost points, we'll need a separate file for each face. 
# These will correspond to an upwinded and a downwinded file for each direction.
AD_ddnD = ixp.declarerank2("AD_ddnD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x0D.h",set_BU_to_print(),params="outCverbose=False")

AD_dupD = ixp.declarerank2("AD_dupD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x0U.h",set_BU_to_print(),params="outCverbose=False")

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x1D.h",set_BU_to_print(),params="outCverbose=False")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x1U.h",set_BU_to_print(),params="outCverbose=False")

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x2D.h",set_BU_to_print(),params="outCverbose=False")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x2U.h",set_BU_to_print(),params="outCverbose=False")


Wrote to file "GiRaFFE_HO/src/B_from_A_10.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_8.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_6.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_4.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x0D.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x0U.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x1D.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x1U.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x2D.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x2U.h"


<a id='etk'></a>

# Step 2: Interfacing with the Einstein Toolkit \[Back to [top](#toc)\]
$$\label{etk}$$


<a id='einstein_c'></a>

## Step 2.a: Constructing the Einstein Toolkit C-code calling functions that include the C code kernels. \[Back to [top](#toc)\]
$$\label{einstein_c}$$

Now that we have generated the C code kernel *GiRaFFE_RHSs.h* and the parameters file *NRPy_params.h*, we will need to write C code to make use of these files. To do this, we can simply follow the example within the [IDScalarWaveNRPy tutorial module](Tutorial-ETK_thorn-IDScalarWaveNRPy.ipynb). Functions defined by these files will be called by the Einstein Toolkit scheduler (specified in schedule.ccl below).

Also, we will write the logic that determines which files are called where in order to calculate $B^i$ here. 

1. Take the primary finite differencing order $N$ from the param.ccl file. Fill in the interior points with the corresponding FD order. 
1. Then, for each face, at $0+{\rm cctk\_nghostzones[face]}-1$ and ${\rm cctk\_lsh[face]}-{\rm cctk\_nghostzones[face]}+1$, calculate $B^i$ at order $N-2$
1. Continue moving outwards: at the points $0+{\rm cctk\_nghostzones[face]}-p$ and ${\rm cctk\_lsh[face]}-{\rm cctk\_nghostzones[face]}+p$, calculate B at order $N-2p$.
1. When ${\rm cctk\_nghostzones[face]}-p = 0$, use the upwinding and downwinding derivatives for the appropriate face.

In [5]:
%%writefile GiRaFFE_HO/src/GiRaFFE.c
#include <math.h>
#include <stdio.h>
#include <stdlib.h>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "Symmetry.h"

const int MAXFACE = -1;
const int NUL     = +0;
const int MINFACE = +1;

/* TODO: Start using this to directly interface with HydroBase!
*CCTK_REAL *Bvec0,*Bvec1,*Bvec2;
*Bvec0 = &Bvec[0*cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]];
*Bvec1 = &Bvec[1*cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]];
*Bvec2 = &Bvec[2*cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]];
*/

// Declare boundary condition FACE_UPDATE function,
// which fills in the ghost zones with successively
// lower order finite differencing
void AtoB(const cGH* restrict const cctkGH,const int ORDER, 
          const CCTK_REAL *AD0GF,const CCTK_REAL *AD1GF,const CCTK_REAL *AD2GF,
          CCTK_REAL *BU0GF,CCTK_REAL *BU1GF,CCTK_REAL *BU2GF,
          const int i0min, const int i0max, 
          const int i1min, const int i1max, 
          const int i2min, const int i2max, 
          const int FACEX0, const int FACEX1, const int FACEX2) {
  
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  const CCTK_REAL *gammaDD00GF = gxx;
  const CCTK_REAL *gammaDD01GF = gxy;
  const CCTK_REAL *gammaDD02GF = gxz;
  const CCTK_REAL *gammaDD11GF = gyy;
  const CCTK_REAL *gammaDD12GF = gyz;
  const CCTK_REAL *gammaDD22GF = gzz;

  if(ORDER==8) {
    printf("Computing A to B with Order = 8...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_8.h"
        if(isnan(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]) || isinf(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)])) {
            printf("i0,i1,i2 = %d,%d,%d\n",i0,i1,i2);
        }
    }
  } else if(ORDER==6) {
    printf("Computing A to B with Order = 6...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_6.h"
        if(isnan(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]) || isinf(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)])) {
            printf("i0,i1,i2 = %d,%d,%d\n",i0,i1,i2);
        }
    }
  } else if(ORDER==4) {
    printf("Computing A to B with Order = 4...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_4.h"
        if(isnan(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]) || isinf(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)])) {
            printf("i0,i1,i2 = %d,%d,%d\n",i0,i1,i2);
        }
    }
  } else if(ORDER==2) {
    printf("Computing A to B with Order = 2...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_2.h"
        if(isnan(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]) || isinf(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)])) {
            printf("i0,i1,i2 = %d,%d,%d\n",i0,i1,i2);
        }
    } 
  } else if(ORDER==0) {
    if(FACEX0==MAXFACE) {
    printf("Computing A to B at x = max...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x0D.h"
            if(isnan(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]) || isinf(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)])) {
                printf("i0,i1,i2 = %d,%d,%d\n",i0,i1,i2);
            }
        }
    } else if(FACEX0==MINFACE) {
    printf("Computing A to B at x = min...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x0U.h"
            if(isnan(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]) || isinf(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)])) {
                printf("i0,i1,i2 = %d,%d,%d\n",i0,i1,i2);
            }
        }
    } else if(FACEX1==MAXFACE) {
    printf("Computing A to B at y = max...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x1D.h"
            if(isnan(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]) || isinf(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)])) {
                printf("i0,i1,i2 = %d,%d,%d\n",i0,i1,i2);
            }
        }
    } else if(FACEX1==MINFACE) {
    printf("Computing A to B at y = min...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x1U.h"
            if(isnan(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]) || isinf(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)])) {
                printf("i0,i1,i2 = %d,%d,%d\n",i0,i1,i2);
            }
        }
    } else if(FACEX2==MAXFACE) {
    printf("Computing A to B at z = max...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x2D.h"
            if(isnan(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]) || isinf(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)])) {
                printf("i0,i1,i2 = %d,%d,%d\n",i0,i1,i2);
            }
        }
    } else if(FACEX2==MINFACE) {
    printf("Computing A to B at z = min...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x2U.h"
            if(isnan(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]) || isinf(BU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)])) {
                printf("i0,i1,i2 = %d,%d,%d\n",i0,i1,i2);
            }
        }
    } else {
        printf("ERROR. FACEX parameters not set properly.\n");
        exit(1);
    }
  } else {
    printf("ERROR. ORDER = %d not supported!\n",ORDER);
    exit(1);
  }
}

void driver_A_to_B(CCTK_ARGUMENTS) {
  
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const int *NG = cctk_nghostzones;
  const int *Nx = cctk_lsh;
  CCTK_INT ORDER = NG[0]*2;
  for(int ii=0;ii<cctk_lsh[2]*cctk_lsh[1]*cctk_lsh[0];ii++) {
      BU0[ii] = 1.0 / 0.0;
      BU1[ii] = 1.0 / 0.0;
      BU2[ii] = 1.0 / 0.0;
  }

  printf("Starting A to B driver with Order = %d...\n",ORDER);
  AtoB(cctkGH,ORDER, AD0,AD1,AD2,BU0,BU1,BU2, NG[0],Nx[0]-NG[0],NG[1],Nx[1]-NG[1],NG[2],Nx[2]-NG[2], NUL,NUL,NUL);
  int imin[3] = { NG[0], NG[1], NG[2] };
  int imax[3] = { Nx[0]-NG[0], Nx[1]-NG[1], Nx[2]-NG[2] };
  while(ORDER>0) {
      // After updating each face, adjust imin[] and imax[] 
      //   to reflect the newly-updated face extents.
      ORDER -= 2;
      AtoB(cctkGH,ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0]-1,imin[0], imin[1],imax[1], imin[2],imax[2], MINFACE,NUL,NUL); 
      if(ORDER!=0) imin[0]--;
      AtoB(cctkGH,ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imax[0],imax[0]+1, imin[1],imax[1], imin[2],imax[2], MAXFACE,NUL,NUL); 
      if(ORDER!=0) imax[0]++;

      AtoB(cctkGH,ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0],imax[0], imin[1]-1,imin[1], imin[2],imax[2], NUL,MINFACE,NUL); 
      if(ORDER!=0) imin[1]--;
      AtoB(cctkGH,ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0],imax[0], imax[1],imax[1]+1, imin[2],imax[2], NUL,MAXFACE,NUL); 
      if(ORDER!=0) imax[1]++;

      AtoB(cctkGH,ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0],imax[0], imin[1],imax[1], imin[2]-1,imin[2], NUL,NUL,MINFACE); 
      if(ORDER!=0) imin[2]--;
      AtoB(cctkGH,ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0],imax[0], imin[1],imax[1], imax[2],imax[2]+1, NUL,NUL,MAXFACE); 
      if(ORDER!=0) imax[2]++;
    }
}

void GiRaFFE_HO_calc_prereqs(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                             const CCTK_REAL invdx0,const CCTK_REAL invdx1,const CCTK_REAL invdx2,
                             const CCTK_REAL *alphaGF, const CCTK_REAL *betaU0GF, const CCTK_REAL *betaU1GF, const CCTK_REAL *betaU2GF,
                             const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                             const CCTK_REAL *ValenciavU0GF, const CCTK_REAL *ValenciavU1GF, const CCTK_REAL *ValenciavU2GF, const CCTK_REAL *u4upperZeroGF,
                             const CCTK_REAL *AD0GF,const CCTK_REAL *AD1GF,const CCTK_REAL *AD2GF,const CCTK_REAL *psi6PhiGF,
                             const CCTK_REAL *BU0GF,const CCTK_REAL *BU1GF,const CCTK_REAL *BU2GF,
                             CCTK_REAL *gammaUU00GF,CCTK_REAL *gammaUU01GF,CCTK_REAL *gammaUU02GF,CCTK_REAL *gammaUU11GF,CCTK_REAL *gammaUU12GF,CCTK_REAL *gammaUU22GF,CCTK_REAL *gammadetGF,
                             CCTK_REAL *SevolParenUD00GF,CCTK_REAL *SevolParenUD01GF,CCTK_REAL *SevolParenUD02GF,
                             CCTK_REAL *SevolParenUD10GF,CCTK_REAL *SevolParenUD11GF,CCTK_REAL *SevolParenUD12GF,
                             CCTK_REAL *SevolParenUD20GF,CCTK_REAL *SevolParenUD21GF,CCTK_REAL *SevolParenUD22GF,
                             CCTK_REAL *AevolParenGF,CCTK_REAL *PevolParenU0GF,CCTK_REAL *PevolParenU1GF,CCTK_REAL *PevolParenU2GF) {

  DECLARE_CCTK_PARAMETERS;

#include "Prereqs.h" 

}

void GiRaFFE_HO_calc_rhs(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                         const CCTK_REAL invdx0,const CCTK_REAL invdx1,const CCTK_REAL invdx2,
                         const CCTK_REAL *alphaGF, const CCTK_REAL *betaU0GF, const CCTK_REAL *betaU1GF, const CCTK_REAL *betaU2GF,
                         const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                         const CCTK_REAL *gammaUU00GF,const CCTK_REAL *gammaUU01GF,const CCTK_REAL *gammaUU02GF,const CCTK_REAL *gammaUU11GF,const CCTK_REAL *gammaUU12GF,const CCTK_REAL *gammaUU22GF,const CCTK_REAL *gammadetGF,
                         const CCTK_REAL *ValenciavU0GF, const CCTK_REAL *ValenciavU1GF, const CCTK_REAL *ValenciavU2GF, const CCTK_REAL *u4upperZeroGF,
                         const CCTK_REAL *AD0GF,const CCTK_REAL *AD1GF,const CCTK_REAL *AD2GF,const CCTK_REAL *psi6PhiGF,
                         const CCTK_REAL *BU0GF,const CCTK_REAL *BU1GF,const CCTK_REAL *BU2GF,
                         const CCTK_REAL *SevolParenUD00GF,const CCTK_REAL *SevolParenUD01GF,const CCTK_REAL *SevolParenUD02GF,
                         const CCTK_REAL *SevolParenUD10GF,const CCTK_REAL *SevolParenUD11GF,const CCTK_REAL *SevolParenUD12GF,
                         const CCTK_REAL *SevolParenUD20GF,const CCTK_REAL *SevolParenUD21GF,const CCTK_REAL *SevolParenUD22GF,
                         const CCTK_REAL *AevolParenGF,const CCTK_REAL *PevolParenU0GF,const CCTK_REAL *PevolParenU1GF,const CCTK_REAL *PevolParenU2GF,
                         CCTK_REAL *Stilde_rhsD0GF, CCTK_REAL *Stilde_rhsD1GF, CCTK_REAL *Stilde_rhsD2GF,
                         CCTK_REAL *A_rhsD0GF, CCTK_REAL *A_rhsD1GF, CCTK_REAL *A_rhsD2GF, CCTK_REAL *psi6Phi_rhsGF) {
  DECLARE_CCTK_PARAMETERS;

#include "Conservs.h"

}

void calc_u0(const CCTK_REAL alpha,const CCTK_INT idx,
             const CCTK_REAL gammaDD00,const CCTK_REAL gammaDD01,const CCTK_REAL gammaDD02,const CCTK_REAL gammaDD11,const CCTK_REAL gammaDD12,const CCTK_REAL gammaDD22,
             CCTK_REAL *ValenciavU0GF,CCTK_REAL *ValenciavU1GF,CCTK_REAL *ValenciavU2GF,CCTK_REAL *u0GF)
{
  DECLARE_CCTK_PARAMETERS;
  CCTK_REAL u0;
  CCTK_REAL ValenciavU0 = ValenciavU0GF[idx];
  CCTK_REAL ValenciavU1 = ValenciavU1GF[idx];
  CCTK_REAL ValenciavU2 = ValenciavU2GF[idx];

#include "computeu0_Cfunction.h"

  u0GF[idx] = u0;
  ValenciavU0GF[idx] = ValenciavU0;
  ValenciavU1GF[idx] = ValenciavU1;
  ValenciavU2GF[idx] = ValenciavU2;
}

void GiRaFFE_HO_set_GFs_to_finite_difference_for_RHS(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  
#pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              const CCTK_INT idx = CCTK_GFINDEX3D(cctkGH, i0,i1,i2);
              calc_u0(alp[idx],idx,
                      gxx[idx],gxy[idx],gxz[idx],gyy[idx],gyz[idx],gzz[idx],
                      ValenciavU0,ValenciavU1,ValenciavU2,u4upperZero);
          }
      }
  }

  GiRaFFE_HO_calc_prereqs(cctkGH,cctk_lsh,cctk_nghostzones,
                          invdx0, invdx1, invdx2,
                          alp, betax, betay, betaz,
                          gxx, gxy, gxz, gyy, gyz, gzz,
                          ValenciavU0, ValenciavU1, ValenciavU2, u4upperZero,
                          AD0, AD1, AD2, psi6Phi,
                          BU0, BU1, BU2,
                          gammaUU00,gammaUU01,gammaUU02,gammaUU11,gammaUU12,gammaUU22,gammadet,
                          SevolParenUD00, SevolParenUD01, SevolParenUD02,
                          SevolParenUD10, SevolParenUD11, SevolParenUD12,
                          SevolParenUD20, SevolParenUD21, SevolParenUD22,
                          AevolParen, PevolParenU0, PevolParenU1, PevolParenU2);
}

void GiRaFFE_HO_set_rhs(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  
  GiRaFFE_HO_calc_rhs(cctkGH,cctk_lsh,cctk_nghostzones,
                      invdx0, invdx1, invdx2,
                      alp, betax, betay, betaz,
                      gxx, gxy, gxz, gyy, gyz, gzz,
                      gammaUU00,gammaUU01,gammaUU02,gammaUU11,gammaUU12,gammaUU22,gammadet,
                      ValenciavU0, ValenciavU1, ValenciavU2, u4upperZero,
                      AD0, AD1, AD2, psi6Phi,
                      BU0, BU1, BU2,
                      SevolParenUD00, SevolParenUD01, SevolParenUD02,
                      SevolParenUD10, SevolParenUD11, SevolParenUD12,
                      SevolParenUD20, SevolParenUD21, SevolParenUD22,
                      AevolParen, PevolParenU0, PevolParenU1, PevolParenU2,
                      Stilde_rhsD0, Stilde_rhsD1, Stilde_rhsD2,
                      A_rhsD0, A_rhsD1, A_rhsD2, psi6Phi_rhs);
}

/* Boundary Condition code adapted from WaveToyC thorn in ETK, implementing built-in
 * ETK BC functionality
 */
void GiRaFFE_HO_SelectBCs(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  const char *bctype;


  bctype = NULL;
  if (CCTK_EQUALS(bound,"flat") || CCTK_EQUALS(bound,"static") ||
      CCTK_EQUALS(bound,"radiation") || CCTK_EQUALS(bound,"robin") ||
      CCTK_EQUALS(bound,"none"))
  {
    bctype = bound;
  }
  else if (CCTK_EQUALS(bound,"zero"))
  {
    bctype = "scalar";
  }

  /* Uses all default arguments, so invalid table handle -1 can be passed */
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::StildeD0", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::StildeD1", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::StildeD2", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::AD0", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::AD1", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::AD2", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::psi6Phi", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
}

void GiRaFFE_HO_InitSymBound(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
      
  int sym[3];

  sym[0] = 1;
  sym[1] = 1;
  sym[2] = 1;

  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::uU0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::uU1");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::uU2");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::u0alpha");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::alpsqrtgam");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::AevolParen");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PevolParenU0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PevolParenU1");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PevolParenU2");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::StildeD0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::StildeD1");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::StildeD2");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::AD0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::AD1");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::AD2");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::psi6Phi");

  return;
}

void GiRaFFE_HO_RegisterVars(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  CCTK_INT ierr CCTK_ATTRIBUTE_UNUSED = 0;
  /* Register all the evolved grid functions with MoL */
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::StildeD0"),  CCTK_VarIndex("GiRaFFE_HO::Stilde_rhsD0"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::StildeD1"),  CCTK_VarIndex("GiRaFFE_HO::Stilde_rhsD1"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::StildeD2"),  CCTK_VarIndex("GiRaFFE_HO::Stilde_rhsD2"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::AD0"),  CCTK_VarIndex("GiRaFFE_HO::A_rhsD0"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::AD1"),  CCTK_VarIndex("GiRaFFE_HO::A_rhsD1"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::AD2"),  CCTK_VarIndex("GiRaFFE_HO::A_rhsD2"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::psi6Phi"),  CCTK_VarIndex("GiRaFFE_HO::psi6Phi_rhs"));
  /* Register all the evolved Array functions with MoL */
  return;
}

Overwriting GiRaFFE_HO/src/GiRaFFE.c


<a id='conservative2primitive'></a>

## Step 2.b: The Conservative to Primitive Solver \[Back to [top](#toc)\]
$$\label{conservative2primitive}$$

We will also need to use the conservative to primitive solver from the old version of $\texttt{GiRaFFE}$, included here for convenience. It is slightly modified to use our new variable names and to otherwise work with the slightly different scheduler. It also computes the inverse and determinant of the three metric.

In [6]:
%%writefile GiRaFFE_HO/src/driver_conserv_to_prims_FFE.C
/* We evolve forward in time a set of functions called the 
 * "conservative variables" (magnetic field and Poynting vector), 
 * and any time the conserv's are updated, we must recover the 
 * primitive variables (velocities), before reconstructing & evaluating 
 * the RHSs of the MHD equations again. 
 *
 * This file contains the routine for this algebraic calculation. 
 * The velocity is calculated with formula (85), arXiv:1310.3274v2
 * $v^i = 4 \pi \alpha \gamma^{ij} {\tilde S}_j \gamma{-1/2} B^{-2} - \beta^i$ 
 * The force-free condition: $B^2>E^2$ is checked before computing the velocity.
 * and after imposing the constraint ${\tilde B}^i {\tilde S}_i = 0$
 
 * The procedure is as described in arXiv:1310.3274v2: 
 * 1. ${\tilde S}_i ->{\tilde S}_i - ({\tilde S}_j {\tilde B}^j) {\tilde B}^i/{\tilde B}^2$
 * 2. $f = \sqrt{(1-\gamma_{max}^{-2}){\tilde B}^4/(16 \pi^2 \gamma {\tilde S}^2)}$ 
 * 3. ${\tilde S}_i -> {\tilde S}_i min(1,f)
 * 4. $v^i = 4 \pi \alpha \gamma^{ij} {\tilde S}_j \gamma{-1/2} B^{-2} - \beta^i$
 * 5. ${\tilde n}_i v^i = 0$
 *
 * All equations are from: http://arxiv.org/pdf/1310.3274.pdf (v2)
 * */

#include "cctk.h"
#include <iostream>
#include <iomanip>
#include <fstream>
#include <sys/time.h>
#include <cmath>
#include <ctime>
#include <cstdlib>
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "Symmetry.h"

#ifndef M_PI
#define M_PI 3.141592653589793238463
#endif

#include "GiRaFFE_headers.h"
//#include "inlined_functions.C"

void GiRaFFE_HO_update_metric_det_inverse(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                                          const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                                          CCTK_REAL *gammaUU00GF,CCTK_REAL *gammaUU01GF,CCTK_REAL *gammaUU02GF,CCTK_REAL *gammaUU11GF,CCTK_REAL *gammaUU12GF,CCTK_REAL *gammaUU22GF,CCTK_REAL *gammadetGF) {

#include "metric_quantities.h"

}

extern "C" void GiRaFFE_HO_conserv_to_prims_FFE(CCTK_ARGUMENTS) {
  printf("Starting conservative-to-primitive solver...\n");
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  // We use proper C++ here, for file I/O later.
  using namespace std;

  const int imin=0,jmin=0,kmin=0;
  const int imax=cctk_lsh[0],jmax=cctk_lsh[1],kmax=cctk_lsh[2];
  
  const CCTK_REAL dz = CCTK_DELTA_SPACE(2);

  CCTK_REAL error_int_numer=0,error_int_denom=0;

  CCTK_INT num_vel_limits=0,num_vel_nulls_current_sheet=0;

  GiRaFFE_HO_update_metric_det_inverse(cctkGH,cctk_lsh,cctk_nghostzones,
                                       gxx, gxy, gxz, gyy, gyz, gzz,
                                       gammaUU00,gammaUU01,gammaUU02,gammaUU11,gammaUU12,gammaUU22,gammadet);

#pragma omp parallel for reduction(+:error_int_numer,error_int_denom,num_vel_limits,num_vel_nulls_current_sheet) schedule(static)
  for(int k=kmin;k<kmax;k++)
    for(int j=jmin;j<jmax;j++)
      for(int i=imin;i<imax;i++) {
        const int index = CCTK_GFINDEX3D(cctkGH,i,j,k);
        
        const CCTK_REAL rL = r[index];
        if(rL>min_radius_inside_of_which_conserv_to_prims_FFE_and_FFE_evolution_is_DISABLED) {

          const CCTK_REAL sqrtg = sqrt(gammadet[index]); // Determinant of 3-metric

          // \gamma_{ij}, computed from \tilde{\gamma}_{ij}
          const CCTK_REAL gxxL = gxx[index];
          const CCTK_REAL gxyL = gxy[index];
          const CCTK_REAL gxzL = gxz[index];
          const CCTK_REAL gyyL = gyy[index];
          const CCTK_REAL gyzL = gyz[index];
          const CCTK_REAL gzzL = gzz[index];

          // \gamma^{ij} = psim4 * \tilde{\gamma}^{ij}
          const CCTK_REAL gupxxL = gammaUU00[index];
          const CCTK_REAL gupxyL = gammaUU01[index];
          const CCTK_REAL gupxzL = gammaUU02[index];
          const CCTK_REAL gupyyL = gammaUU11[index];
          const CCTK_REAL gupyzL = gammaUU12[index];
          const CCTK_REAL gupzzL = gammaUU22[index];

          // Read in magnetic field and momentum variables once from memory, since memory access is expensive:
          const CCTK_REAL BU0L = BU0[index];
          const CCTK_REAL BU1L = BU1[index];
          const CCTK_REAL BU2L = BU2[index];

          // End of page 7 on http://arxiv.org/pdf/1310.3274.pdf
          const CCTK_REAL BtildexL = BU0L*sqrtg;
          const CCTK_REAL BtildeyL = BU1L*sqrtg;
          const CCTK_REAL BtildezL = BU2L*sqrtg;

          const CCTK_REAL Btilde_xL = gxxL*BtildexL + gxyL*BtildeyL + gxzL*BtildezL;
          const CCTK_REAL Btilde_yL = gxyL*BtildexL + gyyL*BtildeyL + gyzL*BtildezL;
          const CCTK_REAL Btilde_zL = gxzL*BtildexL + gyzL*BtildeyL + gzzL*BtildezL;

          CCTK_REAL StildeD0L = StildeD0[index];
          CCTK_REAL StildeD1L = StildeD1[index];
          CCTK_REAL StildeD2L = StildeD2[index];

          if(i==114 && j==114 && k==114) {
              printf("gupxxL = %.15e\nStildeD0L = %.15e\ngupxyL = %.15e\nStildeD1L = %.15e\ngupxzL = %.15e\nStildeD2L = %.15e\n",gupxxL,StildeD0L,gupxyL,StildeD1L,gupxzL,StildeD2L);
          }

          const CCTK_REAL StildeD0_orig = StildeD0L;
          const CCTK_REAL StildeD1_orig = StildeD1L;
          const CCTK_REAL StildeD2_orig = StildeD2L;

          const CCTK_REAL ValenciavU0_orig = ValenciavU0[index];
          const CCTK_REAL ValenciavU1_orig = ValenciavU1[index];
          const CCTK_REAL ValenciavU2_orig = ValenciavU2[index];

          //const CCTK_REAL alpL = alp[index];
          //const CCTK_REAL fourpialpha = 4.0*M_PI*alpL;
          const CCTK_REAL fourpi = 4.0*M_PI;

          //const CCTK_REAL betaxL = betax[index];
          //const CCTK_REAL betayL = betay[index];
          //const CCTK_REAL betazL = betaz[index];

          //* 1. Just below Eq 90: Enforce orthogonality of B^i & S^i, so that B^i S_i = 0
          //*    Correction ${\tilde S}_i ->{\tilde S}_i - ({\tilde S}_j {\tilde B}^j) {\tilde B}_i/{\tilde B}^2$
          //*    NOTICE THAT THE {\tilde B}_i IS LOWERED, AS IT SHOULD BE. THIS IS A TYPO IN PASCHALIDIS ET AL.

          // First compute Btilde^i Stilde_i:
          const CCTK_REAL BtildeiSt_i = StildeD0L*BtildexL + StildeD1L*BtildeyL + StildeD2L*BtildezL;
          //printf("xterm = %f ; yterm = %f ; zterm = %f\n",StildeD0L*BtildexL,StildeD1L*BtildeyL,StildeD2L*BtildezL);

          // Then compute (Btilde)^2
          const CCTK_REAL Btilde2 = gxxL*BtildexL*BtildexL + gyyL*BtildeyL*BtildeyL + gzzL*BtildezL*BtildezL
            + 2.0*(gxyL*BtildexL*BtildeyL + gxzL*BtildexL*BtildezL + gyzL*BtildeyL*BtildezL);

#define APPLY_GRFFE_FIXES

          // Now apply constraint: Stilde_i = Stilde_i - (Btilde^i Stilde_i) / (Btilde)^2
#ifdef APPLY_GRFFE_FIXES
          StildeD0L -= BtildeiSt_i*Btilde_xL/Btilde2;
          StildeD1L -= BtildeiSt_i*Btilde_yL/Btilde2;
          StildeD2L -= BtildeiSt_i*Btilde_zL/Btilde2;
          //printf("BtildeiSt_i = %f ; Btilde2 = %f\n",BtildeiSt_i,Btilde2);
#endif
          // Now that tildeS_i has been fixed, let's compute tildeS^i:
          CCTK_REAL mhd_st_upx = gupxxL*StildeD0L + gupxyL*StildeD1L + gupxzL*StildeD2L;
          CCTK_REAL mhd_st_upy = gupxyL*StildeD0L + gupyyL*StildeD1L + gupyzL*StildeD2L;
          CCTK_REAL mhd_st_upz = gupxzL*StildeD0L + gupyzL*StildeD1L + gupzzL*StildeD2L;

          // Just below Eq. 86 in http://arxiv.org/pdf/1310.3274.pdf:
          CCTK_REAL St2 = StildeD0L*mhd_st_upx + StildeD1L*mhd_st_upy + StildeD2L*mhd_st_upz;

          //* 2. Eq. 92: Factor $f = \sqrt{(1-\gamma_{max}^{-2}){\tilde B}^4/(16 \pi^2 \gamma {\tilde S}^2)}$ 

#ifdef APPLY_GRFFE_FIXES
          const CCTK_REAL gmax = GAMMA_SPEED_LIMIT;
          if(St2 > (1.0 - 1.0/(gmax*gmax))*Btilde2*Btilde2/ (16.0*M_PI*M_PI*sqrtg*sqrtg)) {
            const CCTK_REAL fact = sqrt((1.0 - 1.0/(gmax*gmax))/St2)*Btilde2/(4.0*M_PI*sqrtg);

            //* 3. ${\tilde S}_i -> {\tilde S}_i min(1,f)
            StildeD0L *= MIN(1.0,fact);
            StildeD1L *= MIN(1.0,fact);
            StildeD2L *= MIN(1.0,fact);

            // Recompute S^i
            mhd_st_upx = gupxxL*StildeD0L + gupxyL*StildeD1L + gupxzL*StildeD2L;
            mhd_st_upy = gupxyL*StildeD0L + gupyyL*StildeD1L + gupyzL*StildeD2L;
            mhd_st_upz = gupxzL*StildeD0L + gupyzL*StildeD1L + gupzzL*StildeD2L;
            /*
            printf("%e %e %e | %e %e %e | %e %e %e | oldgamma: %e %e should be > %e vfix\n",x[index],y[index],z[index],
                   BU0L,BU1L,BU2L,
                   St2,(1.0 - 1.0/(gmax*gmax))*Btilde2*Btilde2/ (16.0*M_PI*M_PI*sqrtg*sqrtg),gmax,
                   sqrt(Btilde2 / (Btilde2 - 16*M_PI*M_PI*sqrtg*sqrtg * St2 / Btilde2) ) , Btilde2,16*M_PI*M_PI*sqrtg*sqrtg * St2 / Btilde2  );
            //exit(1);
            */
            // Recompute Stilde^2:
            St2 = StildeD0L*mhd_st_upx + StildeD1L*mhd_st_upy + StildeD2L*mhd_st_upz;

            if( St2 >= Btilde2*Btilde2/ (16.0*M_PI*M_PI*sqrtg*sqrtg) ) {
              printf("ERROR: Velocity cap fix wasn't effective; still have B^2 > E^2\n"); exit(1);
            }
            num_vel_limits++;
          }
#endif

          //* 4. Eq. 85: $v^i = 4 pi \alpha \gamma^{ij} {\tilde S}_j \gamma{-1/2} B^{-2} - \beta^i$: 

          // See, e.g., Eq 71 in http://arxiv.org/pdf/1310.3274.pdf
          // ... or end of page 7 on http://arxiv.org/pdf/1310.3274.pdf:
          const CCTK_REAL B2 = Btilde2/(sqrtg*sqrtg);
          /* 
             Eq. 75: 
             v^i = \alpha \gamma^{ij} S_j / \mathcal{B}^2 - \beta^i
             Eq. 7: \mathcal{B}^{\mu} = B^{\mu}/\sqrt{4 \pi}
             -> v^i = 4 \pi \alpha \gamma^{ij} S_j / B^2 - \beta^i
             Eq. 79: \tilde{S_i} = \sqrt{\gamma} S_i
             -> v^i = 4 \pi \alpha \gamma^{ij} \tilde{S}_j / (\sqrt{\gamma} B^2) - \beta^i
          */
          // Modified from the original GiRaFFE to use Valencia, not drift velocity
          const CCTK_REAL ValenciavU0L = fourpi*mhd_st_upx/(sqrtg*B2);
          const CCTK_REAL ValenciavU1L = fourpi*mhd_st_upy/(sqrtg*B2);
          /* ValenciavU2L not necessarily const! See below. */
          CCTK_REAL ValenciavU2L = fourpi*mhd_st_upz/(sqrtg*B2);
            
          //* 5. Eq. 94: ${\tilde n}_i v^i = 0$ in the current sheet region
          //     n^i is defined as the normal from the current sheet, which lies in the 
          //     xy-plane (z=0). So n = (0,0,1) 
#ifdef APPLY_GRFFE_FIXES
          if(current_sheet_null_v) {
            CCTK_REAL zL = z[index];
            if (fabs(zL) <= (4.0 + 1.0e-2)*dz ) {
              //ValenciavU2L = 0.0;
              ValenciavU2L = - (ValenciavU0L*gxzL + ValenciavU1L*gyzL) / gzzL;
                // FIXME: This is probably not right, but also definitely not the problem. 
            
              // ValenciavU2L reset: TYPICALLY WOULD RESET CONSERVATIVES TO BE CONSISTENT. LET'S NOT DO THAT, TO AVOID MESSING UP B-FIELDS

              if(1==1) {
                CCTK_REAL PRIMS[MAXNUMVARS];
                int ww=0;
                PRIMS[ww] = ValenciavU0L;    ww++;
                PRIMS[ww] = ValenciavU1L;    ww++;
                PRIMS[ww] = ValenciavU2L;    ww++;
                PRIMS[ww] = BU0L;    ww++;
                PRIMS[ww] = BU1L;    ww++;
                PRIMS[ww] = BU2L;    ww++;

                CCTK_REAL METRIC[NUMVARS_FOR_METRIC],dummy=0;
                ww=0;
                // FIXME: NECESSARY?
                //psi_bssn[index] = exp(phi[index]);
                METRIC[ww] = (1.0/12.0) * log(gammadet[index]);ww++;
                METRIC[ww] = dummy;          ww++; // Don't need to set psi.
                METRIC[ww] = gxx[index];    ww++;
                METRIC[ww] = gxy[index];    ww++;
                METRIC[ww] = gxz[index];    ww++;
                METRIC[ww] = gyy[index];    ww++;
                METRIC[ww] = gyz[index];    ww++;
                METRIC[ww] = gzz[index];    ww++;
                METRIC[ww] = alp[index]-1;   ww++;
                METRIC[ww] = betax[index];   ww++;
                METRIC[ww] = betay[index];   ww++;
                METRIC[ww] = betaz[index];   ww++;
                METRIC[ww] = gammaUU00[index];  ww++;
                METRIC[ww] = gammaUU01[index];  ww++;
                METRIC[ww] = gammaUU02[index];  ww++;
                METRIC[ww] = gammaUU11[index];  ww++;
                METRIC[ww] = gammaUU12[index];  ww++;
                METRIC[ww] = gammaUU22[index];  ww++;

                CCTK_REAL CONSERVS[NUM_CONSERVS] = {0.0, 0.0, 0.0}; // 3 conservative variables: Stilde_x, Stilde_y, Stilde_z
                GiRaFFE_HO_compute_conservatives(PRIMS,METRIC, CONSERVS);

                StildeD0L = CONSERVS[STILDEX];
                StildeD1L = CONSERVS[STILDEY];
                StildeD2L = CONSERVS[STILDEZ];
              }
              num_vel_nulls_current_sheet++;
            }
          }
#endif
          ValenciavU0[index] = ValenciavU0L;
          ValenciavU1[index] = ValenciavU1L;      
          ValenciavU2[index] = ValenciavU2L;      
            
          //Now we compute the difference between original & new conservatives, for diagnostic purposes:
          //error_int_numer += fabs(StildeD0L - StildeD0_orig) + fabs(StildeD1L - StildeD1_orig) + fabs(StildeD2L - StildeD2_orig);
          //error_int_denom += fabs(StildeD0_orig) + fabs(StildeD1_orig) + fabs(StildeD2_orig);
          /*
            if(fabs(ValenciavU0_orig) > 1e-13 && fabs(ValenciavU0L-ValenciavU0_orig)/ValenciavU0_orig > 1e-2) printf("BAD ValenciavU0: %e %e | %e %e %e\n",ValenciavU0L,ValenciavU0_orig,x[index],y[index],z[index]);
            if(fabs(ValenciavU1_orig) > 1e-13 && fabs(ValenciavU1L-ValenciavU1_orig)/ValenciavU1_orig > 1e-2) printf("BAD ValenciavU1: %e %e | %e %e %e\n",ValenciavU1L,ValenciavU1_orig,x[index],y[index],z[index]);
            if(fabs(ValenciavU2_orig) > 1e-13 && fabs(ValenciavU2L-ValenciavU2_orig)/ValenciavU2_orig > 1e-2) printf("BAD ValenciavU2: %e %e | %e %e %e\n",ValenciavU2L,ValenciavU2_orig,x[index],y[index],z[index]);
          */
            error_int_numer += fabs(ValenciavU0L - ValenciavU0_orig) + fabs(ValenciavU1L - ValenciavU1_orig) + fabs(ValenciavU2L - ValenciavU2_orig);
            error_int_denom += fabs(ValenciavU0_orig) + fabs(ValenciavU1_orig) + fabs(ValenciavU2_orig);
          


          StildeD0[index] = StildeD0L;
          StildeD1[index] = StildeD1L;
          StildeD2[index] = StildeD2L;
        }
      }

  if (cctk_time==0) {
    /* Write the initial data to separate, dedicated gridfunctions:
       These will be accessed later by VolumeIntegrals_GRMHD to assess convergence. */ 
    #pragma omp parallel for
      for(int i2=0; i2<cctk_lsh[2]; i2++) {
          for(int i1=0; i1<cctk_lsh[1]; i1++) {
              for(int i0=0; i0<cctk_lsh[0]; i0++) {
                  CCTK_INT idx3 = CCTK_GFINDEX3D(cctkGH, i0,i1,i2);
                  AD0_init[idx3] = AD0[idx3];
                  AD1_init[idx3] = AD1[idx3];
                  AD2_init[idx3] = AD2[idx3];
                  psi6Phi_init[idx3] = psi6Phi[idx3];
                  CCTK_REAL lapse = alp[idx3];
                  CCTK_REAL betaxL = betax[idx3];
                  CCTK_REAL betayL = betay[idx3];
                  CCTK_REAL betazL = betaz[idx3];
                  ValenciavU0_init[idx3] = lapse*ValenciavU0[idx3]-betaxL;
                  ValenciavU1_init[idx3] = lapse*ValenciavU1[idx3]-betayL;
                  ValenciavU2_init[idx3] = lapse*ValenciavU2[idx3]-betazL;
                  BU0_init[idx3] = StildeD0[idx3];
                  BU1_init[idx3] = StildeD1[idx3];
                  BU2_init[idx3] = StildeD2[idx3];
              }
          }
      }
  }
  CCTK_VInfo(CCTK_THORNSTRING,"FFEC2P: Lev: %d NumPts= %d | Error: %.3e, ErrDenom: %.3e, v_limits: %d / %d = %.3e, v_nulls: %d / %d = %.3e",
             (int)GetRefinementLevel(cctkGH),
             cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2],
             error_int_numer/(error_int_denom+1e-300),error_int_denom,
             /**/       num_vel_limits,            cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2],
             (CCTK_REAL)num_vel_limits/((CCTK_REAL)cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]),
             /**/       num_vel_nulls_current_sheet,            cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2],
             (CCTK_REAL)num_vel_nulls_current_sheet/((CCTK_REAL)cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]));
}

Overwriting GiRaFFE_HO/src/driver_conserv_to_prims_FFE.C


<a id='old_giraffe'></a>

### Step 2.b.i: Including "GiRaFFE_headers.h" from old version of Giraffe \[Back to [top](#toc)\]
$$\label{old_giraffe}$$

We will also include GiRaFFE_headers.h from the old version of GiRaFFE, which defines constants on which our conservative-to-primitive solver depends.

In [7]:
%%writefile GiRaFFE_HO/src/GiRaFFE_headers.h
// To safeguard against double-including this header file:
#ifndef GIRAFFE_HEADERS_H_
#define GIRAFFE_HEADERS_H_

#define MIN(a,b) ( ((a) < (b)) ? (a) : (b) )
#define MAX(a,b) ( ((a) > (b)) ? (a) : (b) )
#define SQR(x) ((x) * (x))
#define ONE_OVER_SQRT_4PI 0.282094791773878143474039725780

#define VERR_DEF_PARAMS __LINE__, __FILE__, CCTK_THORNSTRING

// The order here MATTERS, as we assume that GUPXX+1=GUPYY, etc.
static const int PHI=0,PSI=1,GXX=2,GXY=3,GXZ=4,GYY=5,GYZ=6,GZZ=7,
  LAPM1=8,SHIFTX=9,SHIFTY=10,SHIFTZ=11,GUPXX=12,GUPYY=13,GUPZZ=14,
  NUMVARS_FOR_METRIC_FACEVALS=15; //<-- Be _sure_ to set this correctly, or you'll have memory access bugs!

// These are not used for facevals in the reconstruction step, but boy are they useful anyway. 
static const int GUPXY=15,GUPXZ=16,GUPYZ=17,
  NUMVARS_FOR_METRIC=18; //<-- Be _sure_ to set this correctly, or you'll have memory access bugs!

// The order here MATTERS, and must be consistent with the order in the in_prims[] array in driver_evaluate_FFE_rhs.C.
static const int VX=0,VY=1,VZ=2,
  BX_CENTER=3,BY_CENTER=4,BZ_CENTER=5,BX_STAGGER=6,BY_STAGGER=7,BZ_STAGGER=8,
  VXR=9,VYR=10,VZR=11,VXL=12,VYL=13,VZL=14,MAXNUMVARS=15;  //<-- Be _sure_ to define MAXNUMVARS appropriately!

static const int UT=0,UX=1,UY=2,UZ=3;

// The "I" suffix denotes interpolation. In other words, these
//    definitions are used for interpolation ONLY. The order here
//    matters as well!
static const int SHIFTXI=0,SHIFTYI=1,SHIFTZI=2,GUPXXI=3,GUPXYI=4,GUPXZI=5,GUPYYI=6,GUPYZI=7,GUPZZI=8,
  PSII=9,LAPM1I=10,A_XI=11,A_YI=12,A_ZI=13,LAPSE_PSI2I=14,LAPSE_OVER_PSI6I=15,MAXNUMINTERP=16;

// Again, the order here MATTERS, since we assume in the code that, e.g., smallb[0]=b^t, smallb[3]=b^z, etc.
static const int SMALLBT=0,SMALLBX=1,SMALLBY=2,SMALLBZ=3,SMALLB2=4,NUMVARS_SMALLB=5;

// Again, the order here MATTERS, since we assume in the code that, CONSERV[STILDEX+1] = \tilde{S}_y
static const int STILDEX=0,STILDEY=1,STILDEZ=2,NUM_CONSERVS=3;

static const int LAPSE=0,PSI2=1,PSI4=2,PSI6=3,PSIM4=4,LAPSEINV=5,NUMVARS_METRIC_AUX=6;
#define SET_LAPSE_PSI4(array_name,METRIC)   {                   \
      array_name[LAPSE] = METRIC[LAPM1]+1.0;                    \
      array_name[PSI2]  = exp(2.0*METRIC[PHI]);                 \
      array_name[PSI4]  = SQR(array_name[PSI2]);                \
      array_name[PSI6]  = array_name[PSI4]*array_name[PSI2];    \
      array_name[PSIM4]  = 1.0/array_name[PSI4];                \
      array_name[LAPSEINV]  = 1.0/array_name[LAPSE];            \
  }

// Keeping track of ghostzones between routines is a nightmare, so
//   we instead attach ghostzone info to each gridfunction and set
//   the ghostzone information correctly within each routine.
struct gf_and_gz_struct {
  CCTK_REAL *gf;
  int gz_lo[4],gz_hi[4];
};

struct output_stats {
  int font_fixed,vel_limited,failure_checker;
  long n_iter;
};


// FIXME: For cosmetic purposes, we might want to make everything either zero-offset or one-offset, instead of a mixture.
const int kronecker_delta[4][3] = { { 0,0,0 },
                                    { 1,0,0 },
                                    { 0,1,0 },
                                    { 0,0,1 } };

/* PUBLIC FUNCTIONS, USED OUTSIDE GiRaFFE AS WELL */
void GiRaFFE_HO_compute_conservatives(const CCTK_REAL *PRIMS,  const CCTK_REAL *METRIC, CCTK_REAL *CONSERVS);
#include "compute_conservatives_FFE.C"

void GiRaFFE_set_symmetry_gzs_staggered(const cGH *cctkGH, const int *cctk_lsh,const CCTK_REAL *X,const CCTK_REAL *Y,const CCTK_REAL *Z, CCTK_REAL *gridfunc,
                                        const CCTK_REAL *gridfunc_syms,const int stagger_x,const int stagger_y,const int stagger_z);

#endif // GIRAFFE_HEADERS_H

Overwriting GiRaFFE_HO/src/GiRaFFE_headers.h


<a id='compute_conservatives'></a>

### Step 2.b.ii: Writing compute_conservatives_FFE.C \[Back to [top](#toc)\]
$$\label{compute_conservatives}$$

The conservative to primitive solver will also depend on the function provided by compute_conservatives_FFE.C.

In [8]:
%%writefile GiRaFFE_HO/src/compute_conservatives_FFE.C
void GiRaFFE_HO_compute_conservatives(const CCTK_REAL *PRIMS,  const CCTK_REAL *METRIC, CCTK_REAL *CONSERVS) {
  const CCTK_REAL psi_bssnL = exp(METRIC[PHI]);
  const CCTK_REAL psi2 = psi_bssnL*psi_bssnL;
  const CCTK_REAL psi4 = psi2*psi2;
  const CCTK_REAL sqrtg = psi4*psi2;

  // \gamma_{ij}, computed from \tilde{\gamma}_{ij}
  const CCTK_REAL gxxL = psi4*METRIC[GXX];
  const CCTK_REAL gxyL = psi4*METRIC[GXY];
  const CCTK_REAL gxzL = psi4*METRIC[GXZ];
  const CCTK_REAL gyyL = psi4*METRIC[GYY];
  const CCTK_REAL gyzL = psi4*METRIC[GYZ];
  const CCTK_REAL gzzL = psi4*METRIC[GZZ];

  // Read in magnetic field and momentum variables once from memory, since memory access is expensive:
  const CCTK_REAL BxL = PRIMS[BX_CENTER];
  const CCTK_REAL ByL = PRIMS[BY_CENTER];
  const CCTK_REAL BzL = PRIMS[BZ_CENTER];

  const CCTK_REAL vxL = PRIMS[VX];
  const CCTK_REAL vyL = PRIMS[VY];
  const CCTK_REAL vzL = PRIMS[VZ];

  //const CCTK_REAL fourpialpha_inv = 1.0/( 4.0*M_PI*(METRIC[LAPM1] + 1.0) );
  const CCTK_REAL fourpi_inv = 1.0/( 4.0*M_PI );

  //const CCTK_REAL betaxL = METRIC[SHIFTX];
  //const CCTK_REAL betayL = METRIC[SHIFTY];
  //const CCTK_REAL betazL = METRIC[SHIFTZ];

  const CCTK_REAL B2 = gxxL*BxL*BxL + gyyL*ByL*ByL + gzzL*BzL*BzL
    + 2.0*(gxyL*BxL*ByL + gxzL*BxL*BzL + gyzL*ByL*BzL);


  // NOTE: SIGNIFICANTLY MODIFIED FROM ILLINOISGRMHD VERSION:
  //       velocities in GiRaFFE are defined to be "drift" velocity.
  //       cf. Eqs 47 and 85 in http://arxiv.org/pdf/1310.3274.pdf 
  // Modified again from the original GiRaFFE to use Valencia velocity

  const CCTK_REAL v_xL = gxxL*vxL + gxyL*vyL + gxzL*vzL;
  const CCTK_REAL v_yL = gxyL*vxL + gyyL*vyL + gyzL*vzL;
  const CCTK_REAL v_zL = gxzL*vxL + gyzL*vyL + gzzL*vzL;
  
  /*
   * Comments:
   * Eq. 85 in https://arxiv.org/pdf/1310.3274.pdf:
   * v^i = 4 pi alpha * (gamma^{ij} tilde{S}_j) / (sqrtgamma * B^2) - beta^i
   * which implies that
   * (v^i + beta^i)*(sqrtgamma * B^2)/(4 pi alpha) = gamma^{ij} tilde{S}_j
   * Multiply both sides by gamma_{ik}:
   * gamma_{ik} (v^i + beta^i)*(sqrtgamma * B^2)/(4 pi alpha) = gamma_{ik} gamma^{ij} tilde{S}_j
   * 
   * -> tilde{S}_k = gamma_{ik} (v^i + beta^i)*(sqrtgamma * B^2)/(4 pi alpha)
   */

  CONSERVS[STILDEX] = v_xL * sqrtg * B2 * fourpi_inv;
  CONSERVS[STILDEY] = v_yL * sqrtg * B2 * fourpi_inv;
  CONSERVS[STILDEZ] = v_zL * sqrtg * B2 * fourpi_inv;
}

Overwriting GiRaFFE_HO/src/compute_conservatives_FFE.C


<a id='grmhd'></a>

## Step 2.c: GRMHD variables provided by HydroBase \[Back to [top](#toc)\]
$$\label{grmhd}$$

This thorn should also use the common GRMHD variables provided by HydroBase, to allow it to interact easily with other MHD thorns. To that end, we will need to read in the common MHD variables at the beginning of our evolution, and then write the new values at the end of our evolution.

In [9]:
%%writefile GiRaFFE_HO/src/GiRaFFE_HydroBase_conversion.c
#include <math.h>
#include <stdio.h>
#include <stdlib.h>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "Symmetry.h"

void HydroBase_to_GiRaFFE(CCTK_ARGUMENTS) {
  /* BUi         <- Bvec[i]
   * ADi         <- Avec[i]
   * ValenciavUi <- vel[i]
   */
  DECLARE_CCTK_PARAMETERS;
  DECLARE_CCTK_ARGUMENTS;

  CCTK_INT idx3;
  CCTK_INT idx4[3];
#pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              idx3 = CCTK_GFINDEX3D(cctkGH, i0,i1,i2);
              idx4[0] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,0);
              idx4[1] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,1);
              idx4[2] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,2);
              BU0[idx3] = Bvec[idx4[0]];
              BU1[idx3] = Bvec[idx4[1]];
              BU2[idx3] = Bvec[idx4[2]];
              AD0[idx3] = Avec[idx4[0]];
              AD1[idx3] = Avec[idx4[1]];
              AD2[idx3] = Avec[idx4[2]];
              ValenciavU0[idx3] = vel[idx4[0]];
              ValenciavU1[idx3] = vel[idx4[1]];
              ValenciavU2[idx3] = vel[idx4[2]];
              // We don't set Phi, because it is always set to zero in GiRaFFE ID.
          }
      }
  }
}

void GiRaFFE_to_HydroBase(CCTK_ARGUMENTS) {
  /* Bvec[i] <- BUi
   * Avec[i] <- ADi
   * vel[i]  <- ValenciavUi
   */
  DECLARE_CCTK_PARAMETERS;
  DECLARE_CCTK_ARGUMENTS;
  
  CCTK_INT idx3;
  CCTK_INT idx4[3];
#pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              idx3 = CCTK_GFINDEX3D(cctkGH, i0,i1,i2);
              idx4[0] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,0);
              idx4[1] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,1);
              idx4[2] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,2);
              Bvec[idx4[0]] = BU0[idx3];
              Bvec[idx4[1]] = BU1[idx3];
              Bvec[idx4[2]] = BU2[idx3];
              Avec[idx4[0]] = AD0[idx3];
              Avec[idx4[1]] = AD1[idx3];
              Avec[idx4[2]] = AD2[idx3];
              vel[idx4[0]] = ValenciavU0[idx3];
              vel[idx4[1]] = ValenciavU1[idx3];
              vel[idx4[2]] = ValenciavU2[idx3];
              // We don't set Phi, because it is always set to zero in GiRaFFE ID thorns.
          }
      }
  }
}

Overwriting GiRaFFE_HO/src/GiRaFFE_HydroBase_conversion.c


<a id='timelevel'></a>

## Step 2.d: Copying initial data to additional timelevels \[Back to [top](#toc)\]
$$\label{timelevel}$$

Since the ETK thorn Boundary will access all three timelevels for our evolved variables, we will need to make sure that those timelevels actually contain data at time zero. So, we will copy initial data to both other timelevels.

In [10]:
%%writefile GiRaFFE_HO/src/postpostinitial__copy_timelevels.c
//-------------------------------------------------
// Stuff to run right after initial data is set up
//-------------------------------------------------

#include "cctk.h"
//#include <cstdio>
//#include <cstdlib>
#include "cctk_Arguments.h"
#include "cctk_Functions.h"
#include "cctk_Parameters.h"
#include "Symmetry.h"
//#include "GiRaFFE_headers.h"

//extern "C" 
void GiRaFFE_HO_PostPostInitial__Copy_Timelevels(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;


  //------------------------------------------------------------------
  // FILL _p AND _p_p TIMELEVELS. Probably don't need to do this if 
  // Carpet::init_fill_timelevels=yes  and
  // MoL::initial_data_is_crap = yes
  // NOTE: We don't fill metric data here.
  // FIXME: Do we really need this?
    if(cctk_time==0) {
#pragma omp parallel for
      for(int k=0;k<cctk_lsh[2];k++) for(int j=0;j<cctk_lsh[1];j++) for(int i=0;i<cctk_lsh[0];i++) {
            const int index = CCTK_GFINDEX3D(cctkGH,i,j,k);

            StildeD0_p[index] = StildeD0[index]; 
            StildeD1_p[index] = StildeD1[index];
            StildeD2_p[index] = StildeD2[index];

            psi6Phi_p[index] = psi6Phi[index];
            AD0_p[index] = AD0[index]; 
            AD1_p[index] = AD1[index];
            AD2_p[index] = AD2[index];

            StildeD0_p_p[index] = StildeD0[index]; 
            StildeD1_p_p[index] = StildeD1[index];
            StildeD2_p_p[index] = StildeD2[index];

            psi6Phi_p_p[index] = psi6Phi[index];
            AD0_p_p[index] = AD0[index]; 
            AD1_p_p[index] = AD1[index];
            AD2_p_p[index] = AD2[index];
          }
    }
}


Overwriting GiRaFFE_HO/src/postpostinitial__copy_timelevels.c


<a id='cclfiles'></a>

## Step 2.e: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure \[Back to [top](#toc)\]
$$\label{cclfiles}$$

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

<a id='interface'></a>

### Step 2.e.i: interface.ccl \[Back to [top](#toc)\]
$$\label{interface}$$

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns. This file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables and functions that exist and are declared "public" within those thorns. Then, we tell the toolkit that we want the gridfunctions $A_i$, $\tilde{S}_i$, and $\sqrt{\gamma}\Phi$ to be visible to other thorns by using the keyword "public". 

In [11]:
%%writefile GiRaFFE_HO/interface.ccl
implements: GiRaFFE_HO

inherits: admbase HydroBase Boundary grid Tmunubase

USES INCLUDE: loopcontrol.h
USES INCLUDE: Symmetry.h
USES INCLUDE: Boundary.h
    
CCTK_INT FUNCTION MoLRegisterEvolved(CCTK_INT IN EvolvedIndex, CCTK_INT IN RHSIndex)
USES FUNCTION MoLRegisterEvolved

CCTK_INT FUNCTION GetBoundarySpecification(CCTK_INT IN size, CCTK_INT OUT ARRAY nboundaryzones, CCTK_INT OUT ARRAY is_internal, CCTK_INT OUT ARRAY is_staggered, CCTK_INT OUT ARRAY shiftout)
USES FUNCTION GetBoundarySpecification

CCTK_INT FUNCTION SymmetryTableHandleForGrid(CCTK_POINTER_TO_CONST IN cctkGH)
USES FUNCTION SymmetryTableHandleForGrid

CCTK_INT FUNCTION Boundary_SelectGroupForBC(CCTK_POINTER_TO_CONST IN GH, CCTK_INT IN faces, CCTK_INT IN boundary_width, CCTK_INT IN table_handle, CCTK_STRING IN group_name, CCTK_STRING IN bc_name)
USES FUNCTION Boundary_SelectGroupForBC

CCTK_INT FUNCTION Boundary_SelectVarForBC(CCTK_POINTER_TO_CONST IN GH, CCTK_INT IN faces, CCTK_INT IN boundary_width, CCTK_INT IN table_handle, CCTK_STRING IN var_name, CCTK_STRING IN bc_name)
USES FUNCTION Boundary_SelectVarForBC

public:
cctk_real GiRaFFE_aux type = GF Timelevels=1 tags='prolongation="none"'
{
  SevolParenUD00,SevolParenUD01,SevolParenUD02,
  SevolParenUD10,SevolParenUD11,SevolParenUD12,
  SevolParenUD20,SevolParenUD21,SevolParenUD22,
  AevolParen,PevolParenU0,PevolParenU1,PevolParenU2,
  gammaUU00,gammaUU01,gammaUU02,gammaUU11,gammaUU12,gammaUU22,gammadet
} "The evolved scalar fields"

public:
cctk_real GiRaFFE_Bs type = GF Timelevels=1 tags='InterpNumTimelevels=1 prolongation="none"'
{
  BU0,BU1,BU2
} "The B field"

public:
cctk_real GiRaFFE_Vs type = GF Timelevels=1 tags='InterpNumTimelevels=1 prolongation="none"'
{
  u4upperZero,ValenciavU0,ValenciavU1,ValenciavU2
} "The zeroth component of the four velocity and the Valencia 3-velocity"

public:
cctk_real GiRaFFE_rhs type = GF Timelevels=1 tags='prolongation="none" Checkpoint="no"'
{
  Stilde_rhsD0,Stilde_rhsD1,Stilde_rhsD2,A_rhsD0,A_rhsD1,A_rhsD2,psi6Phi_rhs
} "The evolved scalar fields"

public:
cctk_real GiRaFFE_vars type = GF Timelevels=3 tags='prolongation="none"'
{
  StildeD0,StildeD1,StildeD2,AD0,AD1,AD2,psi6Phi
} "The evolved scalar fields"

public:
cctk_real GiRaFFEfood_init type = GF Timelevels=1 tags='prolongation="none"'
{
  AD0_init,AD1_init,AD2_init,psi6Phi_init,ValenciavU0_init,ValenciavU1_init,ValenciavU2_init,BU0_init,BU1_init,BU2_init
} "Stores the initial data for later debugging"

#########################################
###  Aliased functions from Carpet    ###
#########################################

CCTK_INT FUNCTION                         \
    GetRefinementLevel                    \
        (CCTK_POINTER_TO_CONST IN cctkGH)
USES FUNCTION GetRefinementLevel

Overwriting GiRaFFE_HO/interface.ccl


<a id='param'></a>

### Step 2.e.ii: param.ccl \[Back to [top](#toc)\]
$$\label{param}$$

2. $\text{param.ccl}$: specifies free parameters within the thorn, enabling them to be set at runtime. It is required to provide allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3). A number of parameters are defined, and more parameters can be easily added in later versions. We also set the number of timelevels we will store in memory.

In [12]:
%%writefile GiRaFFE_HO/param.ccl
shares: MethodOfLines

USES CCTK_INT MoL_Num_Evolved_Vars
USES CCTK_INT MoL_Num_ArrayEvolved_Vars

restricted:
CCTK_INT GiRaFFE_HO_MaxNumEvolvedVars "Number of evolved variables used by this thorn" ACCUMULATOR-BASE=MethodofLines::MoL_Num_Evolved_Vars STEERABLE=RECOVER
{
  7:7 :: "Number of evolved variables used by this thorn"
} 7

restricted:
CCTK_INT GiRaFFE_HO_MaxNumArrayEvolvedVars "Number of Array evolved variables used by this thorn" ACCUMULATOR-BASE=MethodofLines::MoL_Num_ArrayEvolved_Vars STEERABLE=RECOVER
{
  0:0 :: "Number of Array evolved variables used by this thorn"
} 0

restricted:
KEYWORD bound "Type of boundary condition to use"
{
  "flat"      :: "Flat (von Neumann, n grad phi = 0) boundary condition"
  "static"    :: "Static (Dirichlet, dphi/dt=0) boundary condition"
  "radiation" :: "Radiation boundary condition"
  "robin"     :: "Robin (phi(r) = C/r) boundary condition"
  "zero"      :: "Zero (Dirichlet, phi=0) boundary condition"
  "none"      :: "Apply no boundary condition"
} "radiation"

restricted:
CCTK_INT timelevels "Number of active timelevels" STEERABLE=RECOVER
{
  0:3 :: ""
} 3

restricted:
CCTK_REAL xi "The damping factor for the psi6Phi evolution equation"
{
 *:* :: "The damping factor for the psi6Phi evolution equation"
} 0.0

# SPEED LIMIT: Set maximum relativistic gamma factor
# 
REAL GAMMA_SPEED_LIMIT "Maximum relativistic gamma factor. Note the default is much higher than IllinoisGRMHD. (GRFFE can handle higher Lorentz factors)"
{
 1:* :: "Positive > 1, though you'll likely have troubles far above 2000."
} 2000.0

REAL min_radius_inside_of_which_conserv_to_prims_FFE_and_FFE_evolution_is_DISABLED "As parameter suggests, this is the minimum radius inside of which the conservatives-to-primitives solver is disabled. In the Aligned Rotator test, this should be set equal to R_NS_aligned_rotator." STEERABLE=ALWAYS
{
  -1.  :: "disable the conservative-to-primitive solver modification"
  (0:* :: "any positive value"
}  -1.

# Set the drift velocity perpendicular to the current sheet to zero.
BOOLEAN current_sheet_null_v "Shall we null the velocity normal to the current sheet?"
{
} "no" #Necessary for the split monopole


Overwriting GiRaFFE_HO/param.ccl


<a id='schedule'></a>

### Step 2.e.iii: schedule.ccl \[Back to [top](#toc)\]
$$\label{schedule}$$

3. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions. $\text{schedule.ccl}$'s official documentation may be found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

For clarity, we will outline the desired schedule we wish to create in the toolkit:

0. **GiRaFFEfood_HO**
    1. **GiRaFFE_ExactWaldID**
        1. Sets up the vector potential and initial Valencia 3-velocity.
        1. Reads: gammaDD02, gammaDD12, gammaDD22 (Everywhere)
        1. Writes: AD0, AD1, AD2, ValenciavU0, ValenciavU1, ValenciavU2 (Everywhere)
    1. **driver_A_to_B**
        1. Computes the magnetic field from the vector potential everywhere (this function is from **GiRaFFE_HO**).
        1. Reads: gammaDD00, gammaDD01, gammaDD02, gammaDD11, gammaDD12, gammaDD22, AD0, AD1, AD2 (Everywhere)
        1. Writes: BU0, BU1, BU2 (Everywhere)
    1. **StildeD_from_initial_data**
        1. Computes the initial Poynting flux from the initial magnetic field and Valencia 3-velocity data.
        1. Reads: gammaDD00, gammaDD01, gammaDD02, gammaDD11, gammaDD12, gammaDD22, BU0, BU1, BU2, ValenciavU0, ValenciavU1, ValenciavU2 (Everywhere)
        1. Writes: StildeD0, StildeD1, StildeD2 (Interior)
0. **GiRaFFE_HO**
    1. **HydroBase_to_GiRaFFE**
        1. Reads data from **HydroBase** variables into **GiRaFFE_HO** variables.
        1. Reads: Avec, Bvec, vel (from **HydroBase**) (Everywhere)
        1. Writes: AD0, AD1, AD2, BU0, BU1, BU2, ValenciavU0, ValenciavU1, ValenciavU2 (Everywhere)
    1. **GiRaFFE_HO_set_GFs_to_finite_difference_for_RHS**
        1. Sets auxiliary gridfunctions that will need to be finite-differenced for the right-hand sides.
        1. Reads: alpha, betaU0, betaU1, betaU2, gammaDD00, gammaDD01, gammaDD02, gammaDD11, gammaDD12, gammaDD22, ValenciavU0, ValenciavU1, ValenciavU2, AD0, AD1, AD2, psi6Phi (Everywhere)
        1. Writes: uU0, uU1, uU2, u0alpha, alpsqrtgam, AevolParen, PevolParenU0, PevolParenU1, PevolParenU2, gammaUU00, gammaUU01, gammaUU02, gammaUU11, gammaUU12, gammaUU22, gammadet, u4upperZero (Everywhere)
    1. **GiRaFFE_HO_set_rhs**
        1. Sets the RHSs for the ETK's MoL solver.
        1. Reads: alpha, betaU0, betaU1, betaU2, gammaDD00, gammaDD01, gammaDD02, gammaDD11, gammaDD12, gammaDD22, ValenciavU0, ValenciavU1, ValenciavU2, AD0, AD1, AD2, BU0, BU1, BU2, uU0, uU1, uU2, u0alpha, alpsqrtgam, AevolParen, PevolParenU0, PevolParenU1, PevolParenU2, gammaUU00, gammaUU01, gammaUU02, gammaUU11, gammaUU12, gammaUU22, gammadet (Everywhere)
        1. Writes: Stilde_rhsD0, Stilde_rhsD1, Stilde_rhsD2, A_rhsD0, A_rhsD1, A_rhsD2, psi6Phi_rhs (Interior)
    1. **GiRaFFE_HO_SelectBCs**
        1. Apply boundary conditions. 
        1. Reads: StildeD0, StildeD1, StildeD2, AD0, AD1, AD2, psi6Phi (Interior)
        1. Writes: StildeD0, StildeD1, StildeD2, AD0, AD1, AD2, psi6Phi (Boundaries)
    1. **driver_A_to_B**
        1. Computes the magnetic field from the vector potential everywhere.
        1. Reads: gammaDD00, gammaDD01, gammaDD02, gammaDD11, gammaDD12, gammaDD22, AD0, AD1, AD2 (Everywhere)
        1. Writes:BU0, BU1, BU2 (Everywhere)
    1. **GiRaFFE_to_HydroBase**
        1. Reads data from **GiRaFFE_HO** variables into **HydroBase** variables.
        1. Reads: AD0, AD1, AD2, BU0, BU1, BU2, ValenciavU0, ValenciavU1, ValenciavU2 (Everywhere)
        1. Writes: Avec, Bvec, vel (from HydroBase) (Everywhere)

We first assign storage for both scalar gridfunctions, and then specify the standardized ETK "scheduling bins" in which we want each of our thorn's functions to run.


In [13]:
%%writefile GiRaFFE_HO/schedule.ccl
STORAGE: GiRaFFE_rhs[1]
STORAGE: GiRaFFE_vars[3]
STORAGE: GiRaFFE_aux[1]
STORAGE: GiRaFFE_Bs[1]
STORAGE: GiRaFFE_Vs[1]
STORAGE: GiRaFFEfood_init[1]

STORAGE: HydroBase::rho[1],HydroBase::press[1],HydroBase::eps[1],HydroBase::vel[1],HydroBase::Bvec[1],HydroBase::Avec[1],HydroBase::Aphi[1]

# POSTPOSTINITIAL
schedule GROUP GiRaFFE_PostPostInitial at CCTK_POSTPOSTINITIAL before MoL_PostStep after HydroBase_Con2Prim
{
} "HydroBase_Con2Prim in CCTK_POSTPOSTINITIAL sets conserv to prim then outer boundaries (OBs, which are technically disabled). The post OB SYNCs actually reprolongate the conservative variables, making cons and prims INCONSISTENT. So here we redo the con2prim, avoiding the SYNC afterward, then copy the result to other timelevels"

#schedule GiRaFFE_HO_InitSymBound at BASEGRID
#{
#  LANG: C
#  OPTIONS: global
#} "Schedule symmetries"

# Sets the gridfunctions that are needed for RHS; GiRaFFE_HO_set_rhs will need these to be set so it can finite-difference them
schedule GiRaFFE_HO_set_GFs_to_finite_difference_for_RHS as GiRaFFE_HO_set_GFs_to_finite_difference_for_RHS IN MoL_CalcRHS
{
  LANG: C
  READS: admbase::alp(Everywhere)
  READS: admbase::betax(Everywhere)
  READS: admbase::betay(Everywhere)
  READS: admbase::betaz(Everywhere)
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  READS: GiRaFFE_HO::psi6Phi(Everywhere)
  WRITES: GiRaFFE_HO::uU0(Everywhere)
  WRITES: GiRaFFE_HO::uU1(Everywhere)
  WRITES: GiRaFFE_HO::uU2(Everywhere)
  WRITES: GiRaFFE_HO::u0alpha(Everywhere)
  WRITES: GiRaFFE_HO::alpsqrtgam(Everywhere)
  WRITES: GiRaFFE_HO::AevolParen(Everywhere)
  WRITES: GiRaFFE_HO::PevolParenU0(Everywhere)
  WRITES: GiRaFFE_HO::PevolParenU1(Everywhere)
  WRITES: GiRaFFE_HO::PevolParenU2(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU00(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU01(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU02(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU11(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU12(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU22(Everywhere)
  WRITES: GiRaFFE_HO::gammadet(Everywhere)
  WRITES: GiRaFFE_HO::u4upperZero(Everywhere)
  SYNC: GiRaFFE_aux
} "Sets prerequisite quantities for the GiRaFFE right-hand sides"

schedule GiRaFFE_HO_set_rhs as GiRaFFE_HO_Evolution IN MoL_CalcRHS after GiRaFFE_HO_set_GFs_to_finite_difference_for_RHS
{
  LANG: C
  READS: admbase::alp(Everywhere)
  READS: admbase::betax(Everywhere)
  READS: admbase::betay(Everywhere)
  READS: admbase::betaz(Everywhere)
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  READS: GiRaFFE_HO::BU0(Everywhere)
  READS: GiRaFFE_HO::BU1(Everywhere)
  READS: GiRaFFE_HO::BU2(Everywhere)
  READS: GiRaFFE_HO::uU0(Everywhere)
  READS: GiRaFFE_HO::uU1(Everywhere)
  READS: GiRaFFE_HO::uU2(Everywhere)
  READS: GiRaFFE_HO::u0alpha(Everywhere)
  READS: GiRaFFE_HO::alpsqrtgam(Everywhere)
  READS: GiRaFFE_HO::AevolParen(Everywhere)
  READS: GiRaFFE_HO::PevolParenU0(Everywhere)
  READS: GiRaFFE_HO::PevolParenU1(Everywhere)
  READS: GiRaFFE_HO::PevolParenU2(Everywhere)
  READS: GiRaFFE_HO::gammaUU00(Everywhere)
  READS: GiRaFFE_HO::gammaUU01(Everywhere)
  READS: GiRaFFE_HO::gammaUU02(Everywhere)
  READS: GiRaFFE_HO::gammaUU11(Everywhere)
  READS: GiRaFFE_HO::gammaUU12(Everywhere)
  READS: GiRaFFE_HO::gammaUU22(Everywhere)
  READS: GiRaFFE_HO::gammadet(Everywhere)
  WRITES: GiRaFFE_HO::Stilde_rhsD0(Interior)
  WRITES: GiRaFFE_HO::Stilde_rhsD1(Interior)
  WRITES: GiRaFFE_HO::Stilde_rhsD2(Interior)
  WRITES: GiRaFFE_HO::A_rhsD0(Interior)
  WRITES: GiRaFFE_HO::A_rhsD1(Interior)
  WRITES: GiRaFFE_HO::A_rhsD2(Interior)
  WRITES: GiRaFFE_HO::psi6Phi_rhs(Interior)
} "Sets the GiRaFFE right-hand sides"

schedule GiRaFFE_HO_SelectBCs in MoL_PostStep
{
  LANG: C
  OPTIONS: level
  SYNC: GiRaFFE_vars
} "Boundaries of GiRaFFE equations"

schedule GROUP ApplyBCs as GiRaFFE_HO_ApplyBCs in MoL_PostStep after GiRaFFE_HO_SelectBCs
{
  READS: GiRaFFE_HO::AD0(Interior)
  READS: GiRaFFE_HO::AD1(Interior)
  READS: GiRaFFE_HO::AD2(Interior)
  READS: GiRaFFE_HO::psi6Phi(Interior)
  READS: GiRaFFE_HO::ValenciavU0(Interior)
  READS: GiRaFFE_HO::ValenciavU1(Interior)
  READS: GiRaFFE_HO::ValenciavU2(Interior)
  READS: GiRaFFE_HO::StildeD0(Interior)
  READS: GiRaFFE_HO::StildeD1(Interior)
  READS: GiRaFFE_HO::StildeD2(Interior)
  WRITES: GiRaFFE_HO::AD0(Boundary)
  WRITES: GiRaFFE_HO::AD1(Boundary)
  WRITES: GiRaFFE_HO::AD2(Boundary)
  WRITES: GiRaFFE_HO::psi6Phi(Boundary)
  WRITES: GiRaFFE_HO::ValenciavU0(Boundary)
  WRITES: GiRaFFE_HO::ValenciavU1(Boundary)
  WRITES: GiRaFFE_HO::ValenciavU2(Boundary)
  WRITES: GiRaFFE_HO::StildeD0(Boundary)
  WRITES: GiRaFFE_HO::StildeD1(Boundary)
  WRITES: GiRaFFE_HO::StildeD2(Boundary)
} "Apply boundary conditions"


schedule GROUP ApplyBCs as GiRaFFE_HO_ApplyBCs at POSTRESTRICT
{
} "Apply boundary conditions"

schedule driver_A_to_B as driver_A_to_B in HydroBase_Boundaries
{
  LANG: C
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  WRITES: GiRaFFE_HO::BU0(Everywhere)
  WRITES: GiRaFFE_HO::BU1(Everywhere)
  WRITES: GiRaFFE_HO::BU2(Everywhere)
} "Calculates the B-field from the vector potential"

schedule GiRaFFE_HO_RegisterVars in MoL_Register
{
  LANG: C
  OPTIONS: meta
} "Register Variables for MoL"

# Nontrivial primitives solve, for vx,vy,vz:
#schedule GiRaFFE_HO_conserv_to_prims_FFE in MoL_CalcRHS after HydroBase_to_GiRaFFE before GiRaFFE_HO_set_GFs_to_finite_difference_for_RHS
#{
#  LANG: C
#  READS: admbase::gxx(Everywhere)
#  READS: admbase::gxy(Everywhere)
#  READS: admbase::gxz(Everywhere)
#  READS: admbase::gyy(Everywhere)
#  READS: admbase::gyz(Everywhere)
#  READS: admbase::gzz(Everywhere)
#  READS: GiRaFFE_HO::BU0(Everywhere)
#  READS: GiRaFFE_HO::BU1(Everywhere)
#  READS: GiRaFFE_HO::BU2(Everywhere)
#  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
#  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
#  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
#  READS: GiRaFFE_HO::StildeD0(Everywhere)
#  READS: GiRaFFE_HO::StildeD1(Everywhere)
#  READS: GiRaFFE_HO::StildeD2(Everywhere)
#  WRITES: GiRaFFE_HO::ValenciavU0(Everywhere)
#  WRITES: GiRaFFE_HO::ValenciavU1(Everywhere)
#  WRITES: GiRaFFE_HO::ValenciavU2(Everywhere)
#  WRITES: GiRaFFE_HO::StildeD0(Everywhere)
#  WRITES: GiRaFFE_HO::StildeD1(Everywhere)
#  WRITES: GiRaFFE_HO::StildeD2(Everywhere)
#} "Applies the FFE condition B^2>E^2 and recomputes the velocities"

# Schedule this AFTER the evolution as well.
schedule GiRaFFE_HO_conserv_to_prims_FFE in HydroBase_Boundaries AFTER driver_A_to_B
{
  LANG: C
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::BU0(Everywhere)
  READS: GiRaFFE_HO::BU1(Everywhere)
  READS: GiRaFFE_HO::BU2(Everywhere)
  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
  READS: GiRaFFE_HO::StildeD0(Everywhere)
  READS: GiRaFFE_HO::StildeD1(Everywhere)
  READS: GiRaFFE_HO::StildeD2(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU0(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU1(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU2(Everywhere)
  WRITES: GiRaFFE_HO::StildeD0(Everywhere)
  WRITES: GiRaFFE_HO::StildeD1(Everywhere)
  WRITES: GiRaFFE_HO::StildeD2(Everywhere)
} "Applies the FFE condition B^2>E^2 and recomputes the velocities"

schedule HydroBase_to_GiRaFFE IN MoL_CalcRHS before GiRaFFE_HO_set_GFs_to_finite_difference_for_RHS
{
  LANG: C
  READS: HydroBase::Avec(Everywhere)
  READS: HydroBase::Bvec(Everywhere)
  READS: HydroBase::vel(Everywhere)
  WRITES: GiRaFFE_HO::BU0(Everywhere)
  WRITES: GiRaFFE_HO::BU1(Everywhere)
  WRITES: GiRaFFE_HO::BU2(Everywhere)
  WRITES: GiRaFFE_HO::AD0(Everywhere)
  WRITES: GiRaFFE_HO::AD1(Everywhere)
  WRITES: GiRaFFE_HO::AD2(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU0(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU1(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU2(Everywhere)
} "Converts the HydroBase variables to GiRaFFE variables"

schedule GiRaFFE_to_HydroBase AT CCTK_ANALYSIS AFTER ML_BSSN_evolCalcGroup
{
  LANG: C
  READS: GiRaFFE_HO::BU0(Everywhere)
  READS: GiRaFFE_HO::BU1(Everywhere)
  READS: GiRaFFE_HO::BU2(Everywhere)
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
  WRITES: HydroBase::Avec(Everywhere)
  WRITES: HydroBase::Bvec(Everywhere)
  WRITES: HydroBase::vel(Everywhere)
} "Converts the GiRaFFE variables to HydroBase variables"

############################################################
# Schedule Blocks that are run BEFORE the evolution to finish setting up initial data:

schedule driver_A_to_B IN GiRaFFE_ID_Converter as initial_driver_A_to_B before first_initialdata
{
  LANG: C
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  WRITES: GiRaFFE_HO::BU0(Everywhere)
  WRITES: GiRaFFE_HO::BU1(Everywhere)
  WRITES: GiRaFFE_HO::BU2(Everywhere)
} "Calculates the B-field from the vector potential"

schedule GiRaFFE_HO_conserv_to_prims_FFE in GiRaFFE_ID_Converter after first_initialdata
{
  LANG: C
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::BU0(Everywhere)
  READS: GiRaFFE_HO::BU1(Everywhere)
  READS: GiRaFFE_HO::BU2(Everywhere)
  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
  READS: GiRaFFE_HO::StildeD0(Everywhere)
  READS: GiRaFFE_HO::StildeD1(Everywhere)
  READS: GiRaFFE_HO::StildeD2(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU0(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU1(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU2(Everywhere)
  WRITES: GiRaFFE_HO::StildeD0(Everywhere)
  WRITES: GiRaFFE_HO::StildeD1(Everywhere)
  WRITES: GiRaFFE_HO::StildeD2(Everywhere)
} "Applies the FFE condition B^2>E^2 and recomputes the velocities"

# Copy data to other timelevels.
#schedule GiRaFFE_HO_PostPostInitial__Copy_Timelevels in GiRaFFE_PostPostInitial as mhdpostid after initial_driver_A_to_B# after p2c
schedule GiRaFFE_HO_PostPostInitial__Copy_Timelevels in SetTmunu as mhdpostid# after initial_driver_A_to_B 
{
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  READS: GiRaFFE_HO::psi6Phi(Everywhere)
  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
  READS: GiRaFFE_HO::StildeD0(Everywhere)
  READS: GiRaFFE_HO::StildeD1(Everywhere)
  READS: GiRaFFE_HO::StildeD2(Everywhere)
  LANG: C
} "Compute post-initialdata quantities"
# FIXME: This is getting run too many times, even during the evolution!

Overwriting GiRaFFE_HO/schedule.ccl


This yields the following, as output by the toolkit itself, with some extra formatting:

  1. GiRaFFE_HO::GiRaFFE_HO_RegisterVars: [meta] Register Variables for MoL
  1. GROUP GiRaFFE_Initial: Schedule GiRaFFE functions in HydroBase_Initial
    1. GiRaFFEfood_HO::GiRaFFE_Food: Initial data for GiRaFFE
    1. GiRaFFE_HO::initial_driver_A_to_B: Calculates the B-field from the vector potential
    1. GiRaFFE_HO::mhdpostid: Compute post-initialdata quantities
    1. GROUP GiRaFFE_ID_Converter: Translate ET-generated, HydroBase-compatible initial data and convert into variables used by GiRaFFE
      1. GiRaFFEfood_HO::first_initialdata: [local] Convert HydroBase initial data (ID) to ID that GiRaFFE can read.
      1. GiRaFFE_HO::GiRaFFE_HO_SelectBCs: [level] Boundaries of GiRaFFE equations
      1. GROUP GiRaFFE_HO_ApplyBCs: Apply boundary conditions
          1. GiRaFFE_HO::driver_A_to_B: Calculates the B-field from the vector potential
      1. GiRaFFE_HO::GiRaFFE_HO_SelectBCs: [level] Boundaries of GiRaFFE equations
      1. GROUP GiRaFFE_HO_ApplyBCs: Apply boundary conditions
          1. GiRaFFE_HO::driver_A_to_B: Calculates the B-field from the vector potential
    1. GROUP GiRaFFE_PostPostInitial: HydroBase_Con2Prim in CCTK_POSTPOSTINITIAL sets conserv to prim then outer boundaries (OBs, which are technically disabled). The post OB SYNCs actually reprolongate the conservative variables, making cons and prims INCONSISTENT. So here we redo the con2prim, avoiding the SYNC afterward, then copy the result to other timelevels
      1. GiRaFFE_HO::GiRaFFE_HO_SelectBCs: [level] Boundaries of GiRaFFE equations
      1. GROUP GiRaFFE_HO_ApplyBCs: Apply boundary conditions
          1. GiRaFFE_HO::driver_A_to_B: Calculates the B-field from the vector potential
    1. GiRaFFE_HO::GiRaFFE_to_HydroBase: Converts the GiRaFFE variables to HydroBase variables
    1. GiRaFFE_HO::GiRaFFE_HO_SelectBCs: [level] Boundaries of GiRaFFE equations
    1. GROUP GiRaFFE_HO_ApplyBCs: Apply boundary conditions
        1. GiRaFFE_HO::driver_A_to_B: Calculates the B-field from the vector potential
            1. GiRaFFE_HO::HydroBase_to_GiRaFFE: Converts the HydroBase variables to GiRaFFE variables
            1. GiRaFFE_HO::GiRaFFE_HO_set_GFs_to_finite_difference_for_RHS: Sets prerequisite quantities for the GiRaFFE right-hand sides
            1. GiRaFFE_HO::GiRaFFE_HO_Evolution: Sets the GiRaFFE right-hand sides
            1. GiRaFFE_HO::GiRaFFE_HO_SelectBCs: [level] Boundaries of GiRaFFE equations
            1. GROUP GiRaFFE_HO_ApplyBCs: Apply boundary conditions
                1. GiRaFFE_HO::driver_A_to_B: Calculates the B-field from the vector potential
  1. GROUP GiRaFFE_HO_ApplyBCs: Apply boundary conditions
    1. GiRaFFE_HO::GiRaFFE_HO_SelectBCs: [level] Boundaries of GiRaFFE equations
    1. GROUP GiRaFFE_HO_ApplyBCs: Apply boundary conditions
        1. GiRaFFE_HO::driver_A_to_B: Calculates the B-field from the vector potential
  1. GiRaFFE_HO::GiRaFFE_to_HydroBase: Converts the GiRaFFE variables to HydroBase variables
    1. GiRaFFE_HO::GiRaFFE_HO_SelectBCs: [level] Boundaries of GiRaFFE equations
    1. GROUP GiRaFFE_HO_ApplyBCs: Apply boundary conditions
        1. GiRaFFE_HO::driver_A_to_B: Calculates the B-field from the vector potential

<a id='einstein_list'></a>

## Step 2.f: Add the C file to Einstein Toolkit compilation list \[Back to [top](#toc)\]

We will also need $\text{make.code.defn}$, which indicates the list of files that need to be compiled. 
$$\label{einstein_list}$$

In [14]:
%%writefile GiRaFFE_HO/src/make.code.defn
SRCS = GiRaFFE.c driver_conserv_to_prims_FFE.C \
       GiRaFFE_HydroBase_conversion.c \
       postpostinitial__copy_timelevels.c

Overwriting GiRaFFE_HO/src/make.code.defn


<a id='latex_pdf_output'></a>

# Step 3: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-ETK_thorn-GiRaFFE_Higher_Order_v2.pdf](Tutorial-ETK_thorn-GiRaFFE_Higher_Order_v2.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [15]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-ETK_thorn-GiRaFFE_Higher_Order_v2.ipynb
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-GiRaFFE_Higher_Order_v2.tex
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-GiRaFFE_Higher_Order_v2.tex
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-GiRaFFE_Higher_Order_v2.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-ETK_thorn-GiRaFFE_Higher_Order_v2.ipynb to latex
[NbConvertApp] Writing 292250 bytes to Tutorial-ETK_thorn-GiRaFFE_Higher_Order_v2.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
